In [2]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import pandas as pd
import string
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter("enwik9")

Reading enwik9 - 1grams ...
Reading enwik9 - 2grams ...


In [3]:
import gensim
from gensim import corpora, models

In [5]:
import re

In [6]:
# use the emoticons to transform symbol to emotion
from ekphrasis.dicts.emoticons import emoticons
emoticons

{'#-:': '<seallips>',
 '#:': '<seallips>',
 "(':": '<happy>',
 '(((((((:': '<happy>',
 '((((((-:': '<happy>',
 '((((((:': '<happy>',
 '(((((-:': '<happy>',
 '(((((:': '<happy>',
 '((((-:': '<happy>',
 '((((:': '<happy>',
 '(((-:': '<happy>',
 '(((:': '<happy>',
 '((-:': '<happy>',
 '((:': '<happy>',
 "(-':": '<happy>',
 '(-:': '<happy>',
 '(-:0': '<angel>',
 '(-:<': '<devil>',
 '(-:O': '<angel>',
 '(-:o': '<angel>',
 '(-:{': '<devil>',
 '(:': '<happy>',
 '(:0': '<angel>',
 '(:<': '<devil>',
 '(:{': '<devil>',
 '(=': '<happy>',
 '(>_<)': '<sad>',
 '(o:': '<happy>',
 ")':": '<sad>',
 ')))))))):': '<sad>',
 ')))))))-:': '<sad>',
 '))))))):': '<sad>',
 '))))))-:': '<sad>',
 ')))))):': '<sad>',
 ')))))-:': '<sad>',
 '))))):': '<sad>',
 '))))-:': '<sad>',
 ')))):': '<sad>',
 ')))-:': '<sad>',
 '))):': '<sad>',
 '))-:': '<sad>',
 ')):': '<sad>',
 ")-':": '<sad>',
 ')-:': '<sad>',
 '):': '<sad>',
 '*)': '<wink>',
 '*-)': '<wink>',
 '*-:': '<kiss>',
 '*:': '<kiss>',
 '*\\0/*': '<happy>',
 '.-:'

In [7]:
## this corrector is not working quite well. so decide no to use at all. 
from ekphrasis.classes.spellcorrect import SpellCorrector
sp = SpellCorrector(corpus="enwik9") 

Reading enwik9 - 1grams ...


In [8]:
print(sp.correct("guuuy"))

guy


In [2]:
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

## Special Comment
- removed special characters e.g. emoticons
- removed name tags starting from @, including airline names
- remove line space (\n)

## Decide 
- remove tags with #?
- remove numbers?
- remove website? (after http)


## ToDo 
- not : is removed by stop_words
- 

# encoding=utf8  
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [9]:
data = pd.read_csv('./Tweets.csv')
doc_set = data['text']
doc_set = doc_set.tolist()

In [10]:
# remove special characters
## https://stackoverflow.com/questions/42370508/how-to-delete-special-characters-such-as-%C5%92%C3%B0%C5%B8-from-tweets
def AsciiFilter(var):
    temp = ''.join([c for c in var if ord(c) < 128])
    return str(temp)

In [11]:
#doc_set2 = [text.encode(errors='ignore').decode('utf-8') for text in doc_set]
doc_set = [AsciiFilter(text) for text in doc_set]

In [12]:
#tokenizer = RegexpTokenizer(r'\w+')
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
# from ekphrasis.classes.tokenizer import SocialTokenizer
# tokenizer = SocialTokenizer()
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
#p_stemmer = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer("english")

In [13]:
en_stop

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 "can't",
 'cannot',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 "s

In [14]:
doc_set[98]

"@virginamerica What is your phone number. I can't find who to call about a flight reservation."

In [15]:
testdoc = doc_set[98]

In [16]:
negs = [w for w in en_stop if "n't" in w]
negsInto = ["are not", "can not", "could not", "did not", "does not", "do not",
           "had not", "has not", "have not", "is not", "must not", "shall not",
           "should not", "was not", "were not", "will not", "would not"]
negDic = {}
for v,w in zip(negs, negsInto) :
    negDic[v] = w

In [17]:
en_stop.remove('not')
en_stop.remove('no')

In [18]:
def ConvertNeg(lowerdoc) :
    doc = lowerdoc
    for w in lowerdoc.split(' ') :
        if w in negs :
            doc = doc.replace(w,negDic[w])
    return doc

In [19]:
def DeleteLine(lowerdoc) :
    doc = lowerdoc
    if '\n' in lowerdoc :
        doc = doc.replace('\n',' ')
    return doc

In [20]:
def NametagTokenFilter(tokens) :
    temp = [str(word) for word in tokens if not word.startswith('@')]
    return temp

def WebAddressFilter(tokens) :
    temp = [str(word) for word in tokens if not word.startswith("http")]
    return temp
# def NonCharacterFilter(tokens) :
#     temp = [ re.sub("[^a-zA-Z#]", "",word) for word in tokens]
#     return temp

def NumberFilter(tokens) :
    temp = [str(word) for word in tokens if not word.isdigit()]
    return temp
# def SpaceFilter(tokens) :
#     temp = [str(word) for word in tokens if not str(word)==""]
#     return temp
def HashRemover(tokens) :
    temp = [seg.segment(i[1:]).split(" ") if i.startswith('#') else [i] for i in tokens]
    # flatten the list 
    temp = [item for sublist in temp for item in sublist]
    return temp
def TransEmotionFilter(tokens) :
    temp = [emoticons[word][1:-1] if word in emoticons else word for word in tokens]
    return temp
## eliminate all doesn't contains either digits or letters 
def stringPuncFilter(tokens):
    temp = [str(word) for word in tokens if bool(re.search(r'(\d)|([a-z])',str(word)))]
    return temp


In [21]:
doctext = testdoc
# clean and tokenize document string
raw = doctext.lower()
raw = ConvertNeg(raw)
raw = DeleteLine(raw)
print(raw)

### test the spliter to see
raw = "@virginamerica what is your #phonenumber. i can not find who to call about a #flightreservation. :) ..."
tokens = tokenizer.tokenize(raw)
print(tokens)
tokens = NametagTokenFilter(tokens)
tokens = WebAddressFilter(tokens)
tokens = NumberFilter(tokens)
# tokens = NonCharacterFilter(tokens)
# tokens = SpaceFilter(tokens)
print(tokens)


@virginamerica what is your phone number. i can not find who to call about a flight reservation.
['@virginamerica', 'what', 'is', 'your', '#phonenumber', '.', 'i', 'can', 'not', 'find', 'who', 'to', 'call', 'about', 'a', '#flightreservation', '.', ':)', '...']
['what', 'is', 'your', '#phonenumber', '.', 'i', 'can', 'not', 'find', 'who', 'to', 'call', 'about', 'a', '#flightreservation', '.', ':)', '...']


In [22]:
tokens = HashRemover(tokens)
print(tokens)

['what', 'is', 'your', 'phone', 'number', '.', 'i', 'can', 'not', 'find', 'who', 'to', 'call', 'about', 'a', 'flight', 'reservation', '.', ':)', '...']


In [23]:
tokens = TransEmotionFilter(tokens)
print(tokens)

['what', 'is', 'your', 'phone', 'number', '.', 'i', 'can', 'not', 'find', 'who', 'to', 'call', 'about', 'a', 'flight', 'reservation', '.', 'happy', '...']


In [24]:
# remove stop words from tokens
tokens = [i for i in tokens if not i in string.punctuation]
tokens = stringPuncFilter(tokens)
stopped_tokens = [i for i in tokens if not i in en_stop]
print(stopped_tokens)

['phone', 'number', 'can', 'not', 'find', 'call', 'flight', 'reservation', 'happy']


In [25]:

# stem tokens
#stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
stemmed_tokens = [s_stemmer.stem(i) for i in stopped_tokens]
print(stemmed_tokens)
# add tokens to list

['phone', 'number', 'can', 'not', 'find', 'call', 'flight', 'reserv', 'happi']


In [26]:
import string
# list for tokenized documents in loop
texts = []

# loop through document list
for i in range(len(doc_set)):
    doctext = doc_set[i]
    # clean and tokenize document string
    raw = doctext.lower()
    raw = ConvertNeg(raw)
    raw = DeleteLine(raw)
    
    tokens = tokenizer.tokenize(raw)
    tokens = NametagTokenFilter(tokens)
    tokens = WebAddressFilter(tokens)
    tokens = HashRemover(tokens)
    tokens = NumberFilter(tokens)
    tokens = TransEmotionFilter(tokens)
    

    # remove stop words from tokens
    tokens = [i for i in tokens if not i in string.punctuation]
    tokens = stringPuncFilter(tokens)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    #stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    stemmed_tokens = [s_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [27]:
dictionary

In [28]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
#data = [['tom','want','apple'],['tom','love','pear']]
res_array = mlb.fit_transform(texts)

In [29]:
len(res_array)

14640

In [34]:
words_relook = mlb.classes_
words_df = pd.DataFrame(words_relook)
words_df.to_csv('words_dict.csv',index = False)
len(words_relook)

9147

In [36]:
len(words_df)

9147

In [81]:
len(words_relook)

9147

In [86]:
topic_words = [3555,1953,3569,2722,4260,3834,5706,5785,4858,1695,5769,1948,3834,4125,8928,1451,1933,7626,8030,6255,7940,7191,2579,5706,4719,4125,4260,1056,5785,1933]
n = 1
for i in topic_words:
    print(words_relook[i-1]),
    if n%10 == 0:
        print("\n")
    n = n+1
## From the result, the 1st group is    

flight
cancel
flightl
delay
hour
get
no
now
late
book


not
can
get
help
will
bag
call
still
time
plane


thank
servic
custom
no
just
help
hour
airlin
now
call




In [94]:
for i in list([3554,5768,2721,3833,1947,4259,5705,4857,7939,5784]): 
    print(words_relook[i])
print("\n")
for i in [5768,7939,1947,3833,5705,7190,2578,4718,4124,8927]:
    print(words_relook[i])
print("\n")
for i in [1952,3554,3568,5768,3833,1947,5705,4124,4190,4259]:
    print(words_relook[i])
print("\n")
    

flight
not
delay
get
can
hour
no
late
thank
now


not
thank
can
get
no
servic
custom
just
help
will


cancel
flight
flightl
not
get
can
no
help
hold
hour




In [51]:
df = pd.DataFrame(res_array)
df.to_csv('matrix_data.csv',index=False)

In [68]:
df = pd.DataFrame(texts)
df.to_csv('clean_data.csv', index=True)

In [67]:
with open("tweeters_snowball_ver2.txt", "a") as f : 
    for i in range(len(texts)) :
        docID = 'doc'+str(i)
        words = ''
        for w in texts[i] :
            words = words + w + ' '
        f.write(docID + ' ' + words)
        f.write('\n')
        f.write('\n')
    print(i)

14639


In [ ]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

In [ ]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

In [ ]:
texts[7095]

In [18]:
len(texts)

14640

In [23]:
len(texts)

14640

In [32]:
texts[len(texts)-1]

[u'ppl',
 u'need',
 u'know',
 u'mani',
 u'seat',
 u'next',
 u'flight',
 u'plz',
 u'put',
 'us',
 u'standbi',
 u'peopl',
 u'next',
 u'flight']